In [ ]:
# Import dependencies
import pandas as pd
from sklearn import linear_model
from sqlalchemy import create_engine

In [ ]:
#Import functions
def model(df, position, var):
    df = df.dropna()
    df = df[df['pos'] == position]   

    # the model will use different independent variables depending on position
    if position == 'QB':
        X = df[var]
    elif position == 'RB':
        X = df[var]
    elif position == 'WR' or 'TE':
        X = df[var]
    else:
        print('Invalid position entered')
        return

    y = df['next_yr_ppg']
    reg = linear_model.LinearRegression()
    reg.fit(X, y)
    return reg

def testModelAccuracy(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return

    yTest = df['next_yr_ppg']
    results = model.score(XTest, yTest)
    return results

def testModelDifference(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return


    yPred = model.predict(XTest)
    predAndActual = {'Name': df['player'], 'Predicted PPG': yPred,
                     'Actual PPG': df['next_yr_ppg']}

    # creates df from dictionary above
    database = pd.DataFrame(predAndActual)

    # creates a difference column which depicts the difference between the
    # predicted PPG and actual PPG
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals=3)
    database['Difference'] = database['Predicted PPG'] - database['Actual PPG']
    database['Difference'] = database['Difference'].round(decimals=3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)

    return database, meanDiff, medianDiff, meanAbsDiff, medianAbsDiff

def testModel(model, test, train, position, var):
    accuracy = testModelAccuracy(model, train, position, var)
    differences = testModelDifference(model, test, position, var)
    meanDiff = differences[1]
    medDiff = differences[2]
    meanAbsDiff = differences[3]
    medAbsDiff = differences[4]

    print('The accuracy of the {0} model is {1}'.format(position, accuracy))
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medDiff, medAbsDiff))
    print('\n')
    return

def useModel(model, df1, df2, position, var):
    df1.dropna()
    df2.dropna()
    # checks where the Fant Pos is the position given and returns a data frame
    # with only the rows that include said position
    df1 = df1[df1['pos'] == position]
    df2 = df2[df2['pos'] == position]
    df2 = df2[['plid','player','ppr_g']]
    newdf = df1.merge(df2,how='inner',left_on='plid',right_on='plid')

    # the model will use difference parameters based on position
    if position == 'QB':
        X = df1[var]
    elif position == 'RB':
        X = df1[var]
    elif position == 'WR' or 'TE':
        X = df1[var]
    else:
        print('Invalid position entered')
        return
    yPred = model.predict(X)

    # creates new df with the name of player, their position, and their
    # predicted PPG
    databaseDict = {'Name': df1['player'], 'Pos': df1['pos'], 'Predicted PPG': yPred}
    database = pd.DataFrame(databaseDict)
    database = database.merge(df2,how='inner',left_on='Name',right_on='player')
    database = database.drop(columns=['player','plid'])
    database['Difference'] = database['Predicted PPG'] - database['ppr_g']
    database = database.sort_values(by = ['Predicted PPG'], ascending = False)
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals = 3)
    database['Predicted PPR'] = 17 * database['Predicted PPG']
    database['Predicted PPR'] = database['Predicted PPR'].round(decimals = 3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medianDiff, medianAbsDiff))


     # this adds a position rank column to the dataframe
#     posRank = []
#     posRankNum = 1
#     for index, row in newdf.iterrows():
#         posRank.append(posRankNum)
#         posRankNum += 1

#     database['PosRank'] = posRank


    database = database.reset_index(drop = True)

    return database

In [ ]:
# Connect to Database
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'fballfinalproject.c6sg90iemyn2.us-east-2.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'FFForesight5!!' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD 
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}').format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME)
# Create the connection
cnx = create_engine(postgres_str)

In [ ]:
# Read in dataframes and merge
# fantasy = pd.read_sql_query('''SELECT * FROM fantasy;''', cnx)
# teams = pd.read_sql_query('''SELECT * FROM teams;''', cnx)
# advrush = pd.read_sql_query('''SELECT * FROM advrush;''', cnx)
# df = pd.merge(fantasy,teams[['tmid','passrate']],on='tmid', how='inner')
# df = pd.merge(df,advrush,on='plid',how='inner')
# df = df.drop(columns=['player_y', 'tm_y', 'pos_y',
#                       'age_y','G_y', 'gs_y','tmid_y'])
# df = df.rename(columns={'player_x':'player','tm_x':'tm','pos_x':'pos','age_x':'age','G_x':'G','gs_x':'gs','tmid_x':'tmid'})
# df
df = pd.read_sql_query('''SELECT * FROM fantasy;''', cnx)
df

In [ ]:
groups = df.groupby('player')['G'].sum()
groupsdf = pd.DataFrame(groups)
groupsdf

In [ ]:
rsharesdf['rushshare'] = round(rsharesdf['playrag'] / rsharesdf['teamrag'],3)
rsharesdf['targetshare'] = round(rsharesdf['targets'] / rsharesdf['passes'], 3)
rsharesdf

In [ ]:
df = pd.merge(df,rsharesdf[['plid','rushshare','targetshare']],on='plid',how='inner')
df

In [ ]:
# Check column choices
df.columns

In [ ]:
# Select columns for each position (WR + TE joined in recvar)
qbvar = ['age','passatt_g','passyds_g','passtd_g','rushatt_g','rushyds_g',
         'rushyds_att','ppr_g','starter','next_yr_starter']
rbvar = ['age','rushatt_g','rushyds_g','rushyds_att','tgt_g','rec_g',
         'recyds_g','yds_rec','ppr_g','rushshare','targetshare']
recvar = ['age','tgt_g','rec_g','recyds_g','yds_rec','passrate','targetshare']

In [ ]:
# Separate into training, testing, using, and actual data by Yr (Fantasy) or Year (other dataframes)
train = df[df['yr'] <= 2012]
test = df[(df['yr'] >= 2018) & (df['yr'] <= 2019)]
use = df[df['yr'] == 2019]
use = use[use['next_yr_starter'].notna()]
use = use.astype({"next_yr_starter": int})
actuals = df[df['yr'] == 2020]

In [ ]:
# Name model and put in position
RBmodel = model(train, "RB", rbvar)
WRmodel = model(train, "WR", recvar)
TEmodel = model(train, "TE", recvar)
testModel(RBmodel, test, train, "RB", rbvar)
testModel(WRmodel, test, train, "WR", recvar)
testModel(TEmodel, test, train, "TE", recvar)

In [ ]:
# Use model to make predictions and check predictions
RBStats = useModel(RBmodel, use, actuals,'RB', rbvar)
WRStats = useModel(WRmodel, use, actuals, 'WR', recvar)
TEStats = useModel(TEmodel, use, actuals, 'TE', recvar)

In [ ]:
pd.set_option('display.max_rows', None)
RBStats

In [ ]:
RBStats.sort_values(by=['AbsDifference'])

In [ ]:
WRStats

In [ ]:
WRStats.sort_values(by=['AbsDifference'])

In [ ]:
TEStats

In [ ]:
TEStats.sort_values(by=['AbsDifference'])